In [ ]:
!pip install hazm

     |████████████████████████████████| 317kB 4.6MB/s 
     |████████████████████████████████| 235kB 18.3MB/s 
     |████████████████████████████████| 1.4MB 20.9MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154454 sha256=467a4b2627371b9a02c35572b80ac646754d81536433c73106ac2db15c541052
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394488 sha256=2436ef2daf4281cedb9921fd094c4a230fa2e083c35e505cc407e24fef0e5e47
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input, merge, Add, Concatenate, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
from hazm import *
import re

In [ ]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [ ]:
sent = pd.read_csv('./drive/My Drive/PersianSWN.csv', header=None)

In [ ]:
sentiment = {}
for row in sent[0]:
  temp = row.split('\t')
  name = temp[1]
  if name in sentiment:
    sentiment[name][0].append((float(temp[2])) * (float(temp[3])))
    sentiment[name][1].append((float(temp[2])) * (float(temp[4])))
  else:
    sentiment[name] = [[float(temp[2]) * float(temp[3])], [float(temp[2]) * float(temp[4])]]

NameError: ignored

In [ ]:
for name in sentiment:
  temp = sentiment[name]
  sentiment[name] = [np.mean(temp[0]), np.mean(temp[1])]

In [ ]:
print(sentiment['توانا'])

[0.08496527777777779, 0.02597222222222222]


In [ ]:
data = pd.read_csv('./drive/My Drive/data1.csv')

In [ ]:
data.head()

,Text,Sentiment
0,من با خانواده ام 4 شب در هتل داریوش اقامت داشت...,pos
1,من به همراه همسرم و پسر و مادرم شهریور به مدت ...,pos
2,اتاق ما بزرگی اش خوب بود .کیفیت غذای رستوران خ...,pos
3,در تاريخ 23 لغايت 27 مرداد94 به مدت چهار روز ب...,pos
4,سلام من و خانواده ام در مرداد 94 درهتل داریوش ...,pos


In [ ]:
clean_data = {'Text':[], 'Sentiment':[]}
for i in range(len(data['Text'])):
  if(not isinstance(data['Text'][i], float) and len(data['Text'][i].split(" ")) <= 150 and i != 13996):
    clean_data['Text'].append(data['Text'][i])
    clean_data['Sentiment'].append(data['Sentiment'][i])

In [ ]:
clean_data['Test'] = [x.replace(":;،,><؟?.", "") for x in clean_data['Text']]

In [ ]:
for i in range(len(clean_data['Sentiment'])):
  if(clean_data['Sentiment'][i] == 'pos'):
    clean_data['Sentiment'][i] = [1, 0]
  else:
    clean_data['Sentiment'][i] = [0, 1]

In [ ]:
max_features = 1500
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(clean_data['Text'])
X = tokenizer.texts_to_sequences(clean_data['Text'])
X = pad_sequences(X)
# X[:2]

In [ ]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

In [ ]:
Y = []
for x in X:
  temp = []
  for word in x:
    a = reverse_word_map[word]
    temp.append(word)
    if(a in sentiment):
      temp.append(sentiment[a][0])
      temp.append(sentiment[a][1])
    else:
      temp.append(0)
      temp.append(0)
  Y.append(temp)

In [ ]:
X = pad_sequences(Y)

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 146, 128)          192000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 146, 128)          0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 196)               254800    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 394       
Total params: 447,194
Trainable params: 447,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# this is the placeholder tensor for the input sequences
sequence = Input(shape=(X.shape[1],), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size 128
embedded = Embedding(max_features, 128, input_length=X.shape[1])(sequence)

# apply forwards LSTM
forwards = LSTM(30)(embedded)
# apply backwards LSTM
backwards = LSTM(30, go_backwards=True)(embedded)

# concatenate the outputs of the 2 LSTMs
# merged = merge([forwards, backwards], mode='concat', concat_axis=-1)
merged = Concatenate()([forwards, backwards])
after_dp = Dropout(0.5)(merged)
output = Dense(2, activation='sigmoid')(after_dp)

model = Model(inputs=sequence, outputs=output)
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 146)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 146, 128)     192000      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 30)           19080       embedding_1[0][0]                
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 30)           19080       embedding_1[0][0]                
____________________________________________________________________________________________

In [ ]:
Y = clean_data['Sentiment']
Y = np.array(Y)
print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [0 1]]


In [ ]:
print(X_train.shape)

(12046, 146)


In [ ]:
#one direction
batch_size = 128
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1)

Epoch 1/10
95/95 [==============================] - 27s 245ms/step - loss: 0.6598 - accuracy: 0.6212
Epoch 2/10
95/95 [==============================] - 23s 242ms/step - loss: 0.3639 - accuracy: 0.8616
Epoch 3/10
95/95 [==============================] - 23s 244ms/step - loss: 0.2506 - accuracy: 0.9112
Epoch 4/10
95/95 [==============================] - 23s 245ms/step - loss: 0.2070 - accuracy: 0.9267
Epoch 5/10
95/95 [==============================] - 23s 244ms/step - loss: 0.1824 - accuracy: 0.9376
Epoch 6/10
95/95 [==============================] - 23s 247ms/step - loss: 0.1671 - accuracy: 0.9407
Epoch 7/10
95/95 [==============================] - 23s 246ms/step - loss: 0.1547 - accuracy: 0.9461
Epoch 8/10
95/95 [==============================] - 23s 247ms/step - loss: 0.1346 - accuracy: 0.9554
Epoch 9/10
95/95 [==============================] - 23s 246ms/step - loss: 0.1401 - accuracy: 0.9508
Epoch 10/10
95/95 [==============================] - 24s 248ms/step - loss: 0.1186 - accura

In [ ]:
#bi directional
batch_size = 128
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1)

Epoch 1/10


TypeError: ignored

In [ ]:
Y_pred = model.predict(X_test,batch_size = batch_size)

In [ ]:
Y_pred = Y_pred.argmax(axis=-1)

In [ ]:
#one direction
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[1181  123]
 [ 161 1547]]
              precision    recall  f1-score   support

           0       0.88      0.91      0.89      1304
           1       0.93      0.91      0.92      1708

    accuracy                           0.91      3012
   macro avg       0.90      0.91      0.90      3012
weighted avg       0.91      0.91      0.91      3012



In [ ]:
#bi directional
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[1159  125]
 [ 214 1514]]
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1284
           1       0.92      0.88      0.90      1728

    accuracy                           0.89      3012
   macro avg       0.88      0.89      0.89      3012
weighted avg       0.89      0.89      0.89      3012



In [ ]:
# print(Y_pred)
# print(Y_test)
correct = 0
for i in range(len(Y_test)):
  if(Y_pred[i] != Y_test[i][0]):
    correct += 1
print(correct / len(Y_test))

0.8824701195219123


In [ ]:
A = []
for i in range(len(Y_test)):
  if(Y_pred[i] == Y_test[i][0]):
    A.append(i)

In [ ]:
print(A)

[9, 13, 18, 30, 46, 51, 53, 73, 82, 93, 102, 111, 114, 123, 129, 141, 147, 153, 157, 160, 163, 171, 172, 177, 198, 209, 210, 213, 223, 227, 230, 240, 275, 294, 311, 319, 331, 361, 366, 368, 373, 384, 385, 388, 394, 415, 431, 451, 461, 462, 466, 472, 473, 482, 493, 503, 514, 522, 526, 529, 549, 563, 579, 589, 590, 594, 604, 606, 607, 618, 628, 650, 661, 666, 667, 671, 682, 688, 693, 694, 696, 700, 702, 707, 719, 724, 726, 728, 737, 744, 769, 774, 775, 798, 803, 810, 874, 889, 893, 903, 910, 918, 919, 926, 941, 943, 964, 966, 992, 996, 1012, 1014, 1031, 1036, 1037, 1042, 1063, 1064, 1072, 1089, 1094, 1097, 1103, 1104, 1107, 1115, 1116, 1122, 1124, 1128, 1130, 1148, 1205, 1209, 1224, 1237, 1238, 1245, 1255, 1262, 1274, 1286, 1290, 1305, 1307, 1311, 1315, 1321, 1326, 1327, 1333, 1343, 1344, 1347, 1359, 1360, 1384, 1398, 1411, 1412, 1414, 1433, 1446, 1447, 1473, 1479, 1495, 1509, 1512, 1513, 1514, 1523, 1532, 1553, 1565, 1577, 1578, 1589, 1600, 1607, 1608, 1615, 1622, 1628, 1630, 1631, 1638

In [ ]:
print(Y_test[9])
B = []
for j in X_test[9]:
  if(j >= 1):
    B.append(reverse_word_map[j])
print(B)

[0 1]
['سلام', 'همه', 'چيز', 'داخل', 'اتاق', 'بد', 'بود', 'نظافت', 'سرويس', 'بهداشتي', 'خراب', 'بودن', 'توالت', 'اتاق', 'چون', 'نداشت', 'اتاق', 'و', 'رستورانش', 'هم', 'با', 'توجه', 'به', 'هتل', '5', 'ستاره', 'كيفيت', 'نداشت', 'بسيار', 'شلوغ', 'و', 'از', 'آرامش', 'هتل', 'بسيار', 'بسيار', 'بهتر', 'بود']


In [ ]:
# print(A)
# print(X_test[16])
print(Y_test[183])
B = []
for j in X_test[183]:
  if(j >= 1):
    B.append(reverse_word_map[j])
print(B)

[0 1]
['ما', 'به', 'مدت', 'در', 'این', 'هتل', 'بودیم', 'البته', 'این', 'نظر', 'نیست', 'و', 'ما', 'همین', 'نظر', 'را', 'دارند', 'از', 'بابت', 'تمیزی', 'که', 'مهم', 'هم', 'هست', 'خیلی', 'بد', 'بود', 'و', 'وسایل', 'داخل', 'از', 'سرویس', 'خواب', 'تا', 'راحتی', 'در', 'ورود', 'صد', 'درصد', 'پشیمان', 'کننده', 'بود', 'نزدیک', 'بودن', 'ساحل', 'خوب', 'و', 'کارکنان', 'رستوران', 'خوب', 'بودند', 'ساعت', 'صبحانه', '7', '9', 'بود', 'هزینه', 'هتل', 'خوب', 'بود', 'قیمت', 'غذا', 'و', 'مناسب', 'بود', 'و', 'کیفیت', 'متوسط', 'و', 'جالب', 'و', 'سوال', 'من', 'که', '3ستاره', 'نبود', 'یا', 'بود', 'ما', '3ستاره', 'بودیم', 'برای', 'تهیه', 'تفریحی', 'از', 'خود', 'بگیرید', 'اگه', 'راست', 'میگن', 'درکل', 'محیط', 'مناسب', 'و', 'نامناسب', 'بود']


In [ ]:
model.save('./drive/My Drive/')

INFO:tensorflow:Assets written to: ./drive/My Drive/assets


INFO:tensorflow:Assets written to: ./drive/My Drive/assets
